In [1]:
import os 
import requests


pdf_path = "F:\Large_Language_model\Human-Nutrition-2020-Edition-1598491699.pdf"

# download Pdf
if not os.path.exists(pdf_path):
    # print("hello world")

    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
    # print(url)

    # the local filename to save the download file 
    filename = pdf_path

    # send a GET request to the url 
    response = requests.get(url)

    # check if the request was successfull
    if response.status_code == 200:

        #open the file ans save it 
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"[INFO] The file has been download and saved as {filename}")

    else:
        print(f"[INFO] Faild to download the file. Status code: {response.status_code}")

else:
    print(f"file {pdf_path} exists")

file F:\Large_Language_model\Human-Nutrition-2020-Edition-1598491699.pdf exists


we have got a PDf, let's open it!

In [2]:
import fitz
from tqdm.auto import tqdm 

def text_formatter(text: str) -> str:

    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()

    return cleaned_text


def open_and_read_pdf(pdf_path=str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({
            "page_number":page_number,
            "page_char_count":len(text),
            "page_word_count":len(text.split(" ")),
            "page_sentence_count_raw":len(text.split(". ")),
            "page_token_count":len(text) / 4, # 1 token = ~4 character 
            "text":text
        })


    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:10]

0it [00:00, ?it/s]

[{'page_number': 0,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': 1,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': 2,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count_raw': 1,
  'page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'},
 {'page_number': 3,
  'page_char_count': 212,
  'page_word_count': 32,
  'page_sentence_count_raw': 1,
  'page_token_count': 53.0,
  'text': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science 

In [3]:
import random 
random.sample(pages_and_texts, k=3)

[{'page_number': 780,
  'page_char_count': 442,
  'page_word_count': 67,
  'page_sentence_count_raw': 3,
  'page_token_count': 110.5,
  'text': 'http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=420  \xa0 An interactive or media element has been  excluded from this version of the text. You can  view it online here:  http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=420  \xa0 An interactive or media element has been  excluded from this version of the text. You can  view it online here:  http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=420  Discovering Nutrition Facts  |  739'},
 {'page_number': 1060,
  'page_char_count': 1579,
  'page_word_count': 270,
  'page_sentence_count_raw': 12,
  'page_token_count': 394.75,
  'text': 'Food Preservation  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  Food preservation protects consumers from harmful or toxic food.  There are different ways to preserve food. Some are ancient  metho

In [4]:
import pandas as pd 

df = pd.DataFrame(pages_and_texts)
df.head()

page_number  page_char_count  page_word_count  page_sentence_count_raw  \
0            0               29                4                        1   
1            1                0                1                        1   
2            2              320               54                        1   
3            3              212               32                        1   
4            4              797              145                        2   

   page_token_count                                               text  
0              7.25                      Human Nutrition: 2020 Edition  
1              0.00                                                     
2             80.00  Human Nutrition: 2020  Edition  UNIVERSITY OF ...  
3             53.00  Human Nutrition: 2020 Edition by University of...  
4            199.25  Contents  Preface  University of Hawai‘i at Mā...

In [5]:
df.describe().round(2)

page_number  page_char_count  page_word_count  page_sentence_count_raw  \
count      1208.00          1208.00          1208.00                  1208.00   
mean        603.50          1148.59           198.89                     9.97   
std         348.86           560.44            95.75                     6.19   
min           0.00             0.00             1.00                     1.00   
25%         301.75           762.75           134.00                     4.00   
50%         603.50          1232.50           215.00                    10.00   
75%         905.25          1605.25           271.25                    14.00   
max        1207.00          2308.00           429.00                    32.00   

       page_token_count  
count           1208.00  
mean             287.15  
std              140.11  
min                0.00  
25%              190.69  
50%              308.12  
75%              401.31  
max              577.00

In [6]:
from spacy.lang.en import English

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer 
nlp.add_pipe("sentencizer")

# Create document instance as an example
doc = nlp("This is a sentence. This another sentence. I like elephants.")
assert len(list(doc.sents)) == 3

# Print out our sentences split
list(doc.sents)

[This is a sentence., This another sentence., I like elephants.]

In [7]:
pages_and_texts[31]

{'page_number': 31,
 'page_char_count': 521,
 'page_word_count': 99,
 'page_sentence_count_raw': 4,
 'page_token_count': 130.25,
 'text': 'Cheryl Gibby  Cheryl Gibby was born and raised in Hawai‘i and is a wife and  mother of three. She received her BA, MS in Nutritional Sciences,  and PhD in Nutrition from the University of Hawai‘i at Mānoa. She  has served as an instructor for the introductory Nutrition course  at the University of Hawai‘i at Mānoa, and her research interests  include infant and child health, dental and bone health, mobile  health  interventions,  school  nutrition  policies,  and  online  education.  xxxii  |  About the Contributors'}

In [8]:
for item in tqdm(pages_and_texts):

    item["sentences"] = list(nlp(item["text"]).sents)

    # make sure all sentences are string (the default type is Spacy datatype)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])
    

  0%|          | 0/1208 [00:00<?, ?it/s]

In [9]:
random.sample(pages_and_texts, k=1)

[{'page_number': 962,
  'page_char_count': 1674,
  'page_word_count': 287,
  'page_sentence_count_raw': 17,
  'page_token_count': 418.5,
  'text': 'lose an inch or two in height. Also, short-term memory might not  be as keen as it once was.3  Being either underweight or overweight is also a major concern  for the elderly. However, many older adults remain in relatively  good health and continue to be active into their golden years. Good  nutrition is often the key to maintaining health later in life. In  addition, the fitness and nutritional choices made earlier in life set  the stage for continued health and happiness.\xa0 Older adults should  continue to consume nutrient-dense foods and remain physically  active. However, deficiencies are more common after age sixty,  primarily due to reduced intake or malabsorption. The loss of  mobility among frail, homebound elderly adults also impacts their  access to healthy, diverse foods.  Energy and Macronutrients  Due to reductions in lean b

In [10]:
pages_and_texts[2:]

[{'page_number': 2,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count_raw': 1,
  'page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE',
  'sentences': ['Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'],
  'page_sentence_count_spacy': 1},
 {'page_number': 3,
  'page_char_count': 212,
  'page_word_count': 32,
  'page_sentence_count_raw': 1,
  'page_token_count': 53.0,
  'text': 'Human Nutrition: 

In [11]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

page_number  page_char_count  page_word_count  page_sentence_count_raw  \
count      1208.00          1208.00          1208.00                  1208.00   
mean        603.50          1148.59           198.89                     9.97   
std         348.86           560.44            95.75                     6.19   
min           0.00             0.00             1.00                     1.00   
25%         301.75           762.75           134.00                     4.00   
50%         603.50          1232.50           215.00                    10.00   
75%         905.25          1605.25           271.25                    14.00   
max        1207.00          2308.00           429.00                    32.00   

       page_token_count  page_sentence_count_spacy  
count           1208.00                    1208.00  
mean             287.15                      10.32  
std              140.11                       6.30  
min                0.00                       0.00  
25%              190.69                       5.00  
50%              308.12                      10.00  
75%              401.31                      15.00  
max              577.00                      28.00

## Chunking our sentences together

In [12]:
# Define split size to turn groups of sentences into chunks 
num_sentences_chunk_size = 10 

# create a function to split lists of text recursively into chunk size 
# eg. [20] -> [10, 10] or [25] -> [10, 10, 5]

def split_list(input_list: list[str], 
               slice_size: int=num_sentences_chunk_size) -> list[list[str]]:
    return [input_list[i: i+ slice_size] for i in range(0, len(input_list), slice_size)]


test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [13]:
# Loop through pages and texts and split sentences into chunks 
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentences_chunk_size)
    
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [14]:
random.sample(pages_and_texts, k=1)

[{'page_number': 710,
  'page_char_count': 944,
  'page_word_count': 152,
  'page_sentence_count_raw': 8,
  'page_token_count': 236.0,
  'text': 'Dietary supplements containing copper typically range from a few  micrograms to 15 mg. 5  Learning Activities  Technology Note: The second edition of the Human  Nutrition Open Educational Resource (OER) textbook  features interactive learning activities.\xa0 These activities are  available in the web-based textbook and not available in the  downloadable versions (EPUB, Digital PDF, Print_PDF, or  Open Document).  Learning activities may be used across various mobile  devices, however, for the best user experience it is strongly  recommended that users complete these activities using a  desktop or laptop computer and in Google Chrome.  \xa0 5.\xa0Institute of Medicine (US) Panel on Micronutrients.  (2001). Dietary reference intakes for vitamin A, vitamin K,  arsenic, boron, chromium, copper, iodine, iron,  manganese, molybdenum, nickel, silico

In [15]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

page_number  page_char_count  page_word_count  page_sentence_count_raw  \
count      1208.00          1208.00          1208.00                  1208.00   
mean        603.50          1148.59           198.89                     9.97   
std         348.86           560.44            95.75                     6.19   
min           0.00             0.00             1.00                     1.00   
25%         301.75           762.75           134.00                     4.00   
50%         603.50          1232.50           215.00                    10.00   
75%         905.25          1605.25           271.25                    14.00   
max        1207.00          2308.00           429.00                    32.00   

       page_token_count  page_sentence_count_spacy  num_chunks  
count           1208.00                    1208.00     1208.00  
mean             287.15                      10.32        1.53  
std              140.11                       6.30        0.64  
min                0.00                       0.00        0.00  
25%              190.69                       5.00        1.00  
50%              308.12                      10.00        1.00  
75%              401.31                      15.00        2.00  
max              577.00                      28.00        3.00

# spliting each chunk into its own item 

we had like to embed each chunk of sentences into its own numerical representation. 

That will give us a good level of granularity. 

Meaning, we can dive specifically into the text sample that was used in our model. 

In [17]:
import re 

# split each chunk inot its own item 
pages_and_chunks = []
for item in tqdm(pages_and_chunks):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]


        # join the sentences together inot a paragraph-like structure, aka join the list of sentences into one paragraph 
        join_sentences_chunk = "".join(sentence_chunk).replace("  "," ").strip()
        join_sentences_chunk = re.sub(r'\.([A-Z])', r'.\1', join_sentences_chunk)  # ".A" => ". A"

        chunk_dict["sentence_chunk"] = join_sentences_chunk

        # Get some stats on our chunks 
        chunk_dict["chunk_char_count"] = len(join_sentences_chunk)
        chunk_dict["chunk_ward_count"] = len(word for word in join_sentences_chunk.split(" "))
        chunk_dict["chunk_token_count"]  = len(join_sentences_chunk) / 4 # 1 token = ~4 char 

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

0it [00:00, ?it/s]

0

In [18]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts): 
    for sentence_chunk in item["sentence_chunks"]: 
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka join the list of sentences into one paragraph
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" => ". A" (will work for any captial letter)

        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get some stats on our chunks
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 chars

        pages_and_chunks.append(chunk_dict) 

len(pages_and_chunks)

  0%|          | 0/1208 [00:00<?, ?it/s]

1843

In [19]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 986,
  'sentence_chunk': 'Last updated February 16, 2011. The Benefits of Physical Activity | 945',
  'chunk_char_count': 71,
  'chunk_word_count': 12,
  'chunk_token_count': 17.75}]

In [20]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

page_number  chunk_char_count  chunk_word_count  chunk_token_count
count      1843.00           1843.00           1843.00            1843.00
mean        624.38            734.83            112.72             183.71
std         347.79            447.43             71.07             111.86
min           0.00             12.00              3.00               3.00
25%         321.50            315.00             45.00              78.75
50%         627.00            746.00            114.00             186.50
75%         931.00           1118.50            173.00             279.62
max        1207.00           1831.00            297.00             457.75

In [21]:
df.head()

page_number                                     sentence_chunk  \
0            0                      Human Nutrition: 2020 Edition   
1            2  Human Nutrition: 2020 Edition UNIVERSITY OF HA...   
2            3  Human Nutrition: 2020 Edition by University of...   
3            4  Contents Preface University of Hawai‘i at Māno...   
4            5  Lifestyles and Nutrition University of Hawai‘i...   

   chunk_char_count  chunk_word_count  chunk_token_count  
0                29                 4               7.25  
1               308                42              77.00  
2               210                30              52.50  
3               766               114             191.50  
4               941               142             235.25

## filter chunks of text for short chunks 

These chunks may not contain much useful information. 

In [27]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 9.25 | Text: Type 2 diabetes after 804 | Pregnancy
Chunk token count: 20.25 | Text: Published 2002. Accessed December 2, 2017. Pacific Based Dietary Guidelines | 761
Chunk token count: 21.0 | Text: http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=283   Alcohol Metabolism | 441
Chunk token count: 16.0 | Text: Accessed January 20, 2018. 1032 | The Effect of New Technologies
Chunk token count: 21.75 | Text: http:/ /www.health.gov.fj/?page_id=1406. Accessed November 12, 2017. 652 | Introduction


In [35]:
# filter our DataFrame for rows with under 30 token 
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len

[{'page_number': 2,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': 3,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5},
 {'page_number': 4,
  'sentence_chunk': 'Contents Preface University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program xxv About the Contributors University of Hawai‘i at Mānoa Food Science

### Embedding our text chunks 

Embedding are a broad but pawerful concept. 
while humans understand text, machine understand number.


In [41]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

# Create a list of sentences
sentences = ["The Sentence Transformer library provides an easy way to create embeddings.",
             "Sentences can be embedded one by one or in a list.",
             "I like horses!"]

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embedding}")
    print("")

Sentence: The Sentence Transformer library provides an easy way to create embeddings.
Embedding: [-3.44286300e-02  2.95328330e-02 -2.33643539e-02  5.57257496e-02
 -2.19098590e-02 -6.47060527e-03  1.02848671e-02 -6.57803118e-02
  2.29718108e-02 -2.61121094e-02  3.80420424e-02  5.61403073e-02
 -3.68746668e-02  1.52787855e-02  4.37020473e-02 -5.19723520e-02
  4.89479750e-02  3.58107663e-03 -1.29750716e-02  3.54384189e-03
  4.23262566e-02  3.52606587e-02  2.49402728e-02  2.99176984e-02
 -1.99382678e-02 -2.39752326e-02 -3.33366124e-03 -4.30450439e-02
  5.72014190e-02 -1.32517805e-02 -3.54477949e-02 -1.13936048e-02
  5.55561259e-02  3.61094647e-03  8.88527097e-07  1.14026899e-02
 -3.82230096e-02 -2.43546255e-03  1.51314037e-02 -1.32734815e-04
  5.00659980e-02 -5.50876148e-02  1.73444897e-02  5.00959419e-02
 -3.75959203e-02 -1.04463520e-02  5.08322567e-02  1.24861356e-02
  8.67377147e-02  4.64143082e-02 -2.10690200e-02 -3.90251614e-02
  1.99694349e-03 -1.42345466e-02 -1.86794717e-02  2.826694

In [40]:
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [43]:
embeddings[0].shape

(768,)

In [44]:
embedding = embedding_model.encode("My favorite animal is the cow!")
embedding

array([-1.45920590e-02,  8.02744105e-02, -2.35814322e-02, -3.19281891e-02,
        4.08717766e-02,  5.27201071e-02, -6.59842864e-02,  1.63273159e-02,
        1.03795966e-02, -3.25809568e-02, -2.78962553e-02,  5.04058711e-02,
       -3.03209890e-02, -5.52537898e-03, -2.25000782e-03, -3.40672992e-02,
        4.15263437e-02, -6.02299487e-03, -1.18760578e-02,  5.03419675e-02,
       -2.46708933e-02,  4.90849353e-02, -1.78524833e-02, -2.02774983e-02,
       -3.04977763e-02,  8.45100172e-03, -2.10024435e-02, -2.69277282e-02,
        1.77504458e-02,  1.21456785e-02, -5.96181266e-02, -8.12657699e-02,
        3.16369534e-02, -1.59013201e-03,  1.23865732e-06, -8.03154334e-03,
       -3.90659980e-02,  2.38245055e-02,  3.93481478e-02,  2.12704786e-03,
        2.04685498e-02,  4.92231315e-03, -2.50347648e-02,  1.32805826e-02,
        3.23006883e-02,  5.64529933e-02,  4.20428850e-02,  1.70866884e-02,
       -9.11563262e-02,  1.88811794e-02, -3.20623931e-03,  4.00567340e-04,
       -3.64725515e-02,  

In [45]:
%%time 

embedding_model.to("cpu")

# Embed each chunk one by one 
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/1680 [00:00<?, ?it/s]

CPU times: total: 15min 25s
Wall time: 2min 25s


In [46]:
%%time 

embedding_model.to("cuda")

for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/1680 [00:00<?, ?it/s]

CPU times: total: 16.8 s
Wall time: 17.8 s


In [48]:
%%time 

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
text_chunks[419]

CPU times: total: 0 ns
Wall time: 0 ns


'often. • Calm your “sweet tooth” by eating fruits, such as berries or an apple. • Replace sugary soft drinks with seltzer water, tea, or a small amount of 100 percent fruit juice added to water or soda water. The Food Industry: Functional Attributes of Carbohydrates and the Use of Sugar Substitutes In the food industry, both fast-releasing and slow-releasing carbohydrates are utilized to give foods a wide spectrum of functional attributes, including increased sweetness, viscosity, bulk, coating ability, solubility, consistency, texture, body, and browning capacity. The differences in chemical structure between the different carbohydrates confer their varied functional uses in foods. Starches, gums, and pectins are used as thickening agents in making jam, cakes, cookies, noodles, canned products, imitation cheeses, and a variety of other foods. Molecular gastronomists use slow- releasing carbohydrates, such as alginate, to give shape and texture to their fascinating food creations. Add

In [49]:
len(text_chunks)

1680

## converting Embedding 

In [50]:
%%time 

# Embed all texts in batches 
text_chunk_embeddings = embedding_model.encode(
    text_chunks,
    batch_size=32,
    convert_to_tensor=True
)

text_chunk_embeddings

CPU times: total: 6.17 s
Wall time: 5.91 s


tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

### save embedding to file

In [51]:
pages_and_chunks_over_min_token_len[419]

{'page_number': 318,
 'sentence_chunk': 'often. • Calm your “sweet tooth” by eating fruits, such as berries or an apple. • Replace sugary soft drinks with seltzer water, tea, or a small amount of 100 percent fruit juice added to water or soda water. The Food Industry: Functional Attributes of Carbohydrates and the Use of Sugar Substitutes In the food industry, both fast-releasing and slow-releasing carbohydrates are utilized to give foods a wide spectrum of functional attributes, including increased sweetness, viscosity, bulk, coating ability, solubility, consistency, texture, body, and browning capacity. The differences in chemical structure between the different carbohydrates confer their varied functional uses in foods. Starches, gums, and pectins are used as thickening agents in making jam, cakes, cookies, noodles, canned products, imitation cheeses, and a variety of other foods. Molecular gastronomists use slow- releasing carbohydrates, such as alginate, to give shape and texture 

In [52]:
# save embedding to file 
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [53]:
# import saved file and view 

text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df_load.head()

page_number                                     sentence_chunk  \
0            2  Human Nutrition: 2020 Edition UNIVERSITY OF HA...   
1            3  Human Nutrition: 2020 Edition by University of...   
2            4  Contents Preface University of Hawai‘i at Māno...   
3            5  Lifestyles and Nutrition University of Hawai‘i...   
4            6  The Cardiovascular System University of Hawai‘...   

   chunk_char_count  chunk_word_count  chunk_token_count  \
0               308                42              77.00   
1               210                30              52.50   
2               766               114             191.50   
3               941               142             235.25   
4               998               152             249.50   

                                           embedding  
0  [ 6.74242675e-02  9.02281180e-02 -5.09549025e-...  
1  [ 5.52156232e-02  5.92139475e-02 -1.66167393e-...  
2  [ 2.79801730e-02  3.39813828e-02 -2.06426717e-...  
3  [ 6.82566985e-02  3.81275229e-02 -8.46854690e-...  
4  [ 3.30264382e-02 -8.49768892e-03  9.57159605e-...